In [1]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [2]:
import numpy as np
import pandas as pd
import time
import investpy
from fredapi import Fred
import json
from matplotlib import pyplot as plt
import quandl
from tqdm.notebook import tqdm
from datetime import date
import random
import requests
from bs4 import BeautifulSoup
from app.utils.fetch_data import investing_api, alpha_vantage_api
from app.utils.visualization import _plot_data

## We want to visualize the number of stocks that is over the Displayed moving average

### Paid options
- https://site.financialmodelingprep.com/developer/docs/historical-sp-500-companies-api/


### 1. List of SP500 companies in the past
SP500 list from 2007 March 6. <br>
The wiki link used below is a static link, and it is an old version of the SP500 wikipage. <br>

# TRY1 - Wiki
Wiki page changes drastically, and thus hard to identify the S&P500 conposition change

In [469]:
without = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=183114956")
without = BeautifulSoup(without.content, "html.parser")

tables_OLD = without.find_all("table", class_="wikitable")
table_OLD = tables_OLD[0]
# convert html format table into pandas dataframe
without = pd.read_html(str(table_OLD))[0]

In [323]:
w = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=186180303")
w = BeautifulSoup(w.content, "html.parser")

tables_OLD = w.find_all("table", class_="wikitable")
table_OLD = tables_OLD[0]
# convert html format table into pandas dataframe
w = pd.read_html(str(table_OLD))[0]

In [326]:
w

,Ticker symbol,Company,SEC filings,GICS Sector
0,MMM,3M Company,reports,Industrials
1,ABT,Abbott Labs,reports,Health Care
2,ANF,Abercrombie & Fitch Co.,reports,Consumer Discretionary
3,ACE,ACE Limited,reports,Financials
4,ADBE,Adobe Systems,reports,Information Technology
...,...,...,...,...
495,XTO,XTO Energy Inc.,reports,Energy
496,YHOO,Yahoo Inc.,reports,Information Technology
497,YUM,Yum! Brands Inc,reports,Consumer Discretionary
498,ZMH,Zimmer Holdings,reports,Health Care


In [327]:
set(w['Ticker symbol'].tolist()) - set(without['Ticker symbol'].tolist())

{'ACAS',
 'AKAM',
 'ANF',
 'COV',
 'DDR',
 'DFS',
 'EXPD',
 'EXPE',
 'GGP',
 'GME',
 'HST',
 'ICE',
 'JEC',
 'KFT',
 'LUK',
 'MTW',
 'NBL',
 'NYX',
 'POM',
 'RRC',
 'TDC',
 'TEL',
 'TIE',
 'TSO',
 'TSS',
 'WFR',
 'WPO'}

In [328]:
set(without['Ticker symbol'].tolist()) - set(w['Ticker symbol'].tolist())

{'ADCT',
 'ASN',
 'AV',
 'BMET',
 'BOL',
 'DG',
 'DJ',
 'FDC',
 'HCR',
 'HLT',
 'KMI',
 'KSE',
 'MEL',
 'MXIM',
 'NCR',
 'PD',
 'SANM',
 'SLR',
 'SNV',
 'TEK',
 'TIN',
 'TRB',
 'TSG',
 'TXU',
 'UVN'}

---

# TRY2 - Use most recent wiki page and the changes mentioned

In [3]:
URL_OLD = "https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&direction=next&oldid=112162478"
page_OLD = requests.get(URL_OLD)

soup_OLD = BeautifulSoup(page_OLD.content, "html.parser")

In [4]:
tables_OLD = soup_OLD.find_all("table", class_="wikitable")
table_OLD = tables_OLD[0]
# convert html format table into pandas dataframe
sp500_2007 = pd.read_html(str(table_OLD))[0]

In [5]:
sp500_2007.head(2)

,Company,Ticker symbol,SEC filings,Industry
0,3M Company,MMM,reports,Industrial Conglomerates
1,ACE Limited,ACE,reports,Property & Casualty Insurance


### Apply the changes of SP500 company list, and construct the sp500 list for each date

In [6]:
URL_NEW = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
# the fixed version of the sp500 is
# https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=1095558369
# if the wiki page format changes, some of the codes below might not work.
# in that case, use the fixed version of the wiki page
page_NEW = requests.get(URL_NEW)

soup_NEW = BeautifulSoup(page_NEW.content, "html.parser")

In [7]:
changes = pd.read_html(str(soup_NEW.find(id='changes')))[0]
# concat the multi-headers into single header
changes.columns = ['_'.join(i) for i in list(changes.columns)]
# this is the current list of sp500 companies
constituents = pd.read_html(str(soup_NEW.find(id='constituents')))[0]

In [8]:
changes.head(2)

,Date_Date,Added_Ticker,Added_Security,Removed_Ticker,Removed_Security,Reason_Reason
0,"June 21, 2022",KDP,Keurig Dr Pepper,UA/UAA,Under Armour,Market capitalization change.[5]
1,"June 21, 2022",ON,ON Semiconductor,IPGP,IPG Photonics,Market capitalization change.[5]


In [9]:
constituents.head(2)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916


preprocessing `changes` dataframe

In [10]:
# change date format
changes.Date_Date = pd.to_datetime(changes.Date_Date, format="%B %d, %Y")
changes = changes.loc[changes.Date_Date >= '2000-01-01']

In [11]:
print("There are {} changes starting from {} until {}".format(changes.shape[0], 
                                                              changes.Date_Date.min(), 
                                                              changes.Date_Date.max()))

There are 300 changes starting from 2000-06-07 00:00:00 until 2022-06-21 00:00:00


go through the changes

In [12]:
def update_change_data(change_df, new_line, ascending=True):
    """
    update the dataset that has historical changes of s&p500 list
    the new_line should have the same length as the column list
    """
    assert len(new_line)==change_df.shape[1]
    new_line = pd.DataFrame(new_line)
    new_line.Date_Date = pd.to_datetime(new_line.Date_Date, format="%Y-%m-%d")
    new_line = new_line.replace('', np.nan)
    
    change_df = pd.concat([change_df, new_line], ignore_index = True)
    change_df = change_df.sort_values(by="Date_Date", ascending=ascending)
    change_df.reset_index(inplace=True, drop=True)
    return change_df

def update_sp500_list(ls, add_ticker, remove_ticker):
    """
    update the sp500 list
    """
    if not pd.isna(remove_ticker):
        ls.remove(remove_ticker)
    if not pd.isna(add_ticker):
        ls.append(add_ticker)
    return ls

def update_ticker_names(df, old_ticker_compay, old_ticker, new_ticker, ticker_column='Added_Ticker'):
    df.loc[df['Added_Security']==old_ticker_compay, ticker_column] = df.loc[df['Added_Security']==old_ticker_compay].replace(old_ticker, new_ticker)
    return df  


- The `S&P500 composition changes` are based on the old tickers
- Thus, the ticker change will not be reflected on the `S&P500 composition changes`, but only on the final list of S&P500 list
- The data can be called via apis, using the latest ticker
- Thus, need to update the tickers to the latest names

In [13]:
# need to make some adjustments to the 'changes' dataset

# UAA
changes.Removed_Ticker = changes.Removed_Ticker.str.replace("UA/UAA", "UA")
UAA_change = {
        "Date_Date":["2022-06-21"], 
        "Added_Ticker":'',
        "Added_Security":'',
        "Removed_Ticker":'UAA',
        "Removed_Security":'Under Armour',
        "Reason_Reason":'Market capitalization change.[5]'
    }
changes = update_change_data(changes, UAA_change, False)

adjustments = [
    ['QuintilesIMS', 'Q', 'IQV'],
    ['Willis Towers Watson', 'WLTW', 'WTW'],
    ['Discovery Communications', 'DISCK', 'WBD'],
    ['Facebook', 'FB', 'META'],
    ['Michael Kors', 'KORS', 'CPRI'],
    ['Delphi Automotive', 'DLPH', 'APTV'],
    ['Ingersoll-Rand', 'IR', 'TT'],
    ['Priceline.com', 'PCLN', 'BKNG'],
    ['Harris Corporation', 'HRS', 'LHX'],
    ['Cabot Oil & Gas', 'COG', 'CTRA'],
    ['Jacobs Engineering Group', 'JEC', 'J'],
    ['Tesoro Corporation', 'TSO', 'ANDV'],
    ['Leucadia National', 'LUK', 'JEF'],
    ['Kraft Foods', 'KFT', 'MDLZ'],
    ['Countrywide Credit Industries', 'CFC', 'MDLZ']
]

for (old_ticker_compay, old_ticker, new_ticker) in adjustments:
    changes = update_ticker_names(changes, old_ticker_compay, old_ticker, new_ticker, ticker_column='Added_Ticker')

In [14]:
# initial sp500 list
sp500 = dict()
sp500[str(date.today())] = [constituents.Symbol.tolist()]

# update the list and add them to the sp500 dictionary
# we will go backwards
sp500_list = constituents.Symbol.tolist()

# print("First")
# print(len(set(sp500_list)))

for i, (d, new, _, old, _, _) in changes.iterrows():
    
    try:
        sp500_list = update_sp500_list(sp500_list, old, new)
        sp500[str(d).split(' ')[0]] = [sp500_list.copy()]
        
#         print(str(d).split(' ')[0])
#         print(len(set(sp500[str(d).split(' ')[0]][0])))
    
    except Exception as e:
        print("ERROR:{}. Trying to add {} and remove {}".format(d, old, new))
        print(e)
        
        break
        
#     print("LENGTH of 2022 06 21 : ", len(sp500['2022-06-21'][0]))
# #     if  != 503:
# #         print(old, new)

In [15]:
sp500 = pd.DataFrame(sp500).T
sp500.reset_index(inplace=True)
sp500.columns = ['date', 'tickers']
sp500['date'] = pd.to_datetime(sp500['date'])

In [16]:
print("History of S&P500 ticker list is updated from {} to {}".format(sp500.date.min(), sp500.date.max()))

History of S&P500 ticker list is updated from 2000-06-07 00:00:00 to 2022-07-08 00:00:00


In [17]:
sp500.head(2)

,date,tickers
0,2022-07-08,"[MMM, AOS, ABT, ABBV, ABMD, ACN, ATVI, ADM, AD..."
1,2022-06-21,"[MMM, AOS, ABT, ABBV, ABMD, ACN, ATVI, ADM, AD..."


### Extract the dates each ticker was included in the SP500 index

In [18]:
ticker_list = set()

for i in sp500.tickers:
    ticker_tmp_list = set(i)
    ticker_list.update(ticker_tmp_list)

print("There are {} unique tickers that has been or are in sp500".format(len(ticker_list)))

There are 780 unique tickers that has been or are in sp500


In [19]:
# identify which date each ticker was included in sp500

ticker_dict = dict()
today = date.today().strftime("%Y-%m-%d")

# for each ticker, create a column where it indicates whether it is in the ticker list
# 1 if it is, 0 if not
# use this column to indicate the min and max date of the stock

for t in tqdm(ticker_list):
    found = list()
    for i in range(sp500.shape[0]):
        found.append(len(np.where(np.array(sp500['tickers'][i]) == t)[0]))
    sp500['found'] = found
    
    if sp500.loc[sp500['found']!=0, 'date'].max() == sp500['date'].max():
        max_date = today
    else:
        max_date = sp500.loc[sp500['found']!=0, 'date'].max().strftime("%Y-%m-%d")
        
    ticker_dict[t] = {'min_date' : sp500.loc[sp500['found']!=0, 'date'].min().strftime("%Y-%m-%d"),
                      'max_date' : max_date,
                      'dates' : sp500.loc[sp500['found']!=0, 'date'].astype(str).values.tolist()}

  0%|          | 0/780 [00:00<?, ?it/s]

In [20]:
print("Min and max date of UnderArmour: {}, {}".format(ticker_dict['UA']['min_date'], 
                                                       ticker_dict['UA']['max_date']))

Min and max date of UnderArmour: 2016-04-18, 2022-06-21


In [21]:
print("Min and max date of Apple: {}, {}".format(ticker_dict['AAPL']['min_date'], 
                                                       ticker_dict['AAPL']['max_date']))

Min and max date of Apple: 2000-06-07, 2022-07-08


### Collect data - first trial

In [22]:
# retrieve data for each ticker, but 1 year before the min_date
# this is to make sure we have all the data to calculate the moving averages

if 'stocks' in locals():
    if stocks.shape[0]>0:
        pass
    else:
        stocks = pd.DataFrame()
else:
    stocks = pd.DataFrame()

if 'NA_tickers' in locals():
    pass
else:
    NA_tickers = list()
    
    
for k in tqdm(ticker_dict.keys()):
    
    # set min date as 1 year before the actual min date
    s = ticker_dict[k]['min_date'].split('-')
    s[0] = str(int(s[0])-1)
    start_date = '-'.join(s)
    end_date = ticker_dict[k]['max_date']

    try:
        df = investing_api('stock', k, start_date, end_date)
        time.sleep(random.randrange(5, 10)) # set to pause for a bit when calling an api
    except Exception as e:
        if "ERR#0018" in str(e):
            try:
                # retrieve data from alpha vantage
                df = alpha_vantage_api('stock', k)
                time.sleep(random.randrange(5, 10)) # set to pause for a bit when calling an api
            except:
                NA_tickers.append(k)
                print("Error when calling data for {}".format(k))

        stocks = pd.concat([stocks, df], axis=0)

  0%|          | 0/780 [00:00<?, ?it/s]

2022-07-08 19:52:40,143 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: BAC, from 07/06/1999 to 08/07/2022
2022-07-08 19:52:48,718 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: ENDP, from 12/03/2014 to 02/03/2017
2022-07-08 19:52:56,215 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: AVY, from 07/06/1999 to 08/07/2022
2022-07-08 19:53:05,858 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FOSL, from 23/04/2011 to 05/01/2016
2022-07-08 19:53:14,109 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: TGNA, from 07/06/1999 to 02/06/2017
2022-07-08 19:53:20,836 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: DVA, from 07/06/1999 to 08/07/2022


Error when calling data for ACE


2022-07-08 19:53:32,922 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: PPG, from 07/06/1999 to 08/07/2022
2022-07-08 19:53:41,678 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: TXT, from 07/06/1999 to 08/07/2022
2022-07-08 19:53:50,481 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: ILMN, from 01/12/2014 to 08/07/2022
2022-07-08 19:53:57,578 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: BMS, from 07/06/1999 to 05/12/2014
2022-07-08 19:54:07,266 [app.utils.fetch_data] [INFO] Fetching stock from alpha vantage api: VNT
2022-07-08 19:54:13,889 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: ETSY, from 07/10/2019 to 08/07/2022
2022-07-08 19:54:21,465 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: TIF, from 07/06/1999 to 07/01/2021
2022-07-08 19:54:28,277 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: PFE, from 07/06/1999 to 08/07/2022


Error when calling data for MI


...

2022-07-08 21:53:04,020 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: TSCO, from 21/03/2013 to 08/07/2022
2022-07-08 21:53:14,666 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: ETN, from 07/06/1999 to 08/07/2022
2022-07-08 21:53:27,741 [app.utils.fetch_data] [INFO] Fetching stock from alpha vantage api: ESRX
2022-07-08 21:53:35,933 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FSLR, from 07/06/1999 to 20/03/2017


Error when calling data for APC


2022-07-08 21:53:44,587 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: CTVA, from 07/06/2018 to 08/07/2022
2022-07-08 21:53:53,353 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: SYF, from 19/11/2014 to 08/07/2022
2022-07-08 21:54:00,602 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: MDLZ, from 02/07/2006 to 08/07/2022
2022-07-08 21:54:07,366 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: RJF, from 04/04/2016 to 08/07/2022
2022-07-08 21:54:15,365 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: ACN, from 23/09/2010 to 08/07/2022
2022-07-08 21:54:22,316 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: PM, from 07/06/1999 to 08/07/2022
2022-07-08 21:54:29,933 [app.utils.fetch_data] [INFO] Fetching stock from alpha vantage api: MON
2022-07-08 21:54:39,918 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: CHD, from 05/01/2015 to 08/07/2022
2022-07-08 21:54:47,695 [app.utils

Error when calling data for COL


2022-07-08 21:55:27,429 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: POOL, from 09/10/2019 to 08/07/2022
2022-07-08 21:55:37,119 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: ABMD, from 05/06/2017 to 08/07/2022
2022-07-08 21:55:44,144 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FHN, from 07/06/1999 to 21/06/2013


In [23]:
stocks.head()

,Date,Open,High,Low,Close,Volume,Currency,ticker,type,p_key
0,1999-06-07 00:00:00,5.0,5.0,4.96,5.0,1106400,USD,DVA,stock,1999_06_07_DVA
1,1999-06-08 00:00:00,5.0,5.06,4.94,4.96,649200,USD,DVA,stock,1999_06_08_DVA
2,1999-06-09 00:00:00,4.96,4.98,4.83,4.9,1590900,USD,DVA,stock,1999_06_09_DVA
3,1999-06-10 00:00:00,4.92,4.92,4.81,4.87,876900,USD,DVA,stock,1999_06_10_DVA
4,1999-06-11 00:00:00,4.87,5.02,4.87,5.0,1242900,USD,DVA,stock,1999_06_11_DVA


In [165]:
stocks.to_csv("app/data_storage/S&P500/sp500_history.csv", index=False)

### make additional adjustments on the tickers that does not work

Need to update `sp500` data set and `ticker_dict` dictionary.

In [162]:
ticker_adjustments = dict()

ticker_adjustments['ACE'] = {"new_ticker":'CB',
                             "reason":"Ticker changed"}
ticker_adjustments['MI'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['LEH'] = {"new_ticker":"",
                            "reason":"Bankrupt"}
ticker_adjustments['SMS'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['GAS'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['TE'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['GENZ'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['ABS'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['TLAB'] = {"new_ticker":"",
                            "reason":"Defunct"}
ticker_adjustments['ARG'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['MOLX'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['ANR'] = {"new_ticker":"",
                            "reason":"Defunct"}
ticker_adjustments['NSM'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['SCG'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['MIL'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['SGP'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['RX'] = {"new_ticker":"",
                            "reason":"Gone private"}
ticker_adjustments['ABK'] = {"new_ticker":"AMBC",
                            "reason":"Ticker changed"}
ticker_adjustments['NYX'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['CVC'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['AV'] = {"new_ticker":"",
                            "reason":"Gone private"}
ticker_adjustments['DJ'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['CAM'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['TRB'] = {"new_ticker":"",
                            "reason":"Gone private"}
ticker_adjustments['EK'] = {"new_ticker":"KODK",
                            "reason":"Acquired"}
ticker_adjustments['CEPH'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['FNM'] = {"new_ticker":"FNMA",
                            "reason":"Ticker changed"}
ticker_adjustments['FRX'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['SBL'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['CFC'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['RHT'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['TYC'] = {"new_ticker":"JCI",
                            "reason":"Merged"}
ticker_adjustments['GR'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['SLE'] = {"new_ticker":"",
                            "reason":"Defunct"}
ticker_adjustments['RDC'] = {"new_ticker":"",
                            "reason":"Defunct"} # https://stockinvest.us/stock-price/RDC
ticker_adjustments['GLK'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['BMC'] = {"new_ticker":"",
                            "reason":"Gone private"}
ticker_adjustments['WIN'] = {"new_ticker":"WINMQ",
                            "reason":"Ticker changed"}
ticker_adjustments['SLR'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['PTV'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['CBE'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['KSE'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['FRE'] = {"new_ticker":"FMCC",
                            "reason":"Ticker changed"}
ticker_adjustments['QTRN'] = {"new_ticker":"",
                            "reason":"Gone private"}
ticker_adjustments['BF.B'] = {"new_ticker":"BF-B",
                            "reason":"Misspelled"}
ticker_adjustments['WFR'] = {"new_ticker":"SUNE",
                            "reason":"Ticker changed"}
ticker_adjustments['NOVL'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['CCE'] = {"new_ticker":"",
                            "reason":"Merged"}
ticker_adjustments['MEE'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['TEG'] = {"new_ticker":"",
                            "reason":"Defunct"}
ticker_adjustments['ESV'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['RSH'] = {"new_ticker":"",
                            "reason":"Defunct"}
ticker_adjustments['BS'] = {"new_ticker":"",
                            "reason":"Bankrupt"}
ticker_adjustments['APC'] = {"new_ticker":"",
                            "reason":"Acquired"}
ticker_adjustments['COL'] = {"new_ticker":"",
                            "reason":"Acquired"}

In [163]:
ticker_adjustments

{'ACE': {'new_ticker': 'CB', 'reason': 'Ticker changed'},
 'MI': {'new_ticker': '', 'reason': 'Acquired'},
 'LEH': {'new_ticker': '', 'reason': 'Bankrupt'},
 'SMS': {'new_ticker': '', 'reason': 'Acquired'},
 'GAS': {'new_ticker': '', 'reason': 'Acquired'},
 'TE': {'new_ticker': '', 'reason': 'Acquired'},
 'GENZ': {'new_ticker': '', 'reason': 'Acquired'},
 'ABS': {'new_ticker': '', 'reason': 'Acquired'},
 'TLAB': {'new_ticker': '', 'reason': 'Defunct'},
 'ARG': {'new_ticker': '', 'reason': 'Acquired'},
 'MOLX': {'new_ticker': '', 'reason': 'Acquired'},
 'ANR': {'new_ticker': '', 'reason': 'Defunct'},
 'NSM': {'new_ticker': '', 'reason': 'Acquired'},
 'SCG': {'new_ticker': '', 'reason': 'Acquired'},
 'MIL': {'new_ticker': '', 'reason': 'Acquired'},
 'SGP': {'new_ticker': '', 'reason': 'Acquired'},
 'RX': {'new_ticker': '', 'reason': 'Gone private'},
 'ABK': {'new_ticker': 'AMBC', 'reason': 'Ticker changed'},
 'NYX': {'new_ticker': '', 'reason': 'Acquired'},
 'CVC': {'new_ticker': '', 're

In [160]:
changes.loc[changes.Removed_Ticker=='COL']

,Date_Date,Added_Ticker,Added_Security,Removed_Ticker,Removed_Security,Reason_Reason
71,2018-12-03,LW,Lamb Weston,COL,Rockwell Collins,UTX acquired COL[52]


In [135]:
constituents.loc[constituents.Symbol=="BF.B"]

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
80,BF.B,Brown–Forman,reports,Consumer Staples,Distillers & Vintners,"Louisville, Kentucky",1982-10-31,14693,1870


In [136]:
changes.loc[changes.Added_Ticker=='BF.B']

,Date_Date,Added_Ticker,Added_Security,Removed_Ticker,Removed_Security,Reason_Reason


In [158]:
ticker_dict['BS']

{'min_date': '2000-06-07',
 'max_date': '2000-12-05',
 'dates': ['2000-12-05', '2000-07-27', '2000-06-12', '2000-06-07']}

In [34]:
with open('app/keys/keys.json', 'r') as key_file:
    keys = json.load(key_file)
    
key = keys['alpha_vantage']

In [145]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&apikey={}&outputsize=full'.format("SUNE", key)
r = requests.get(url)
data2 = r.json()

In [146]:
data2['Meta Data']

{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'SUNE',
 '3. Last Refreshed': '2016-04-21',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

In [148]:
pd.DataFrame(data2['Time Series (Daily)'])

,2016-04-21,2016-04-20,2016-04-19,2016-04-18,2016-04-15,2016-04-14,2016-04-13,2016-04-12,2016-04-11,2016-04-08,...,1999-11-12,1999-11-11,1999-11-10,1999-11-09,1999-11-08,1999-11-05,1999-11-04,1999-11-03,1999-11-02,1999-11-01
1. open,0.3611,0.3450,0.3202,0.3100,0.4600,0.6000,0.3900,0.4100,0.4070,0.4510,...,14.8100,13.7500,13.0600,12.7500,13.7500,14.2500,14.6200,15.1200,15.0000,14.5600
2. high,0.3760,0.4100,0.3600,0.3780,0.5400,0.7500,0.3970,0.4302,0.4070,0.4510,...,15.1900,14.5000,13.5600,13.3800,13.9400,14.5000,14.8800,15.1200,15.0000,15.2500
3. low,0.3300,0.3177,0.3100,0.2700,0.3540,0.5500,0.3600,0.3900,0.3640,0.3505,...,14.5000,13.7500,13.0600,12.7500,13.6200,13.6900,14.0000,14.5000,14.3100,14.2500
4. close,0.3370,0.3372,0.3155,0.3440,0.3700,0.5851,0.3701,0.4000,0.3927,0.3629,...,14.5600,14.1900,13.5000,13.0000,13.7500,14.5000,14.2500,14.6900,14.8800,15.0600
5. volume,4537812,69775322,37873620,47288187,135257334,140019513,29794309,40904837,36595947,50473386,...,301900,117500,91100,141000,46500,127000,106500,66300,167900,164700


In [144]:
investpy.get_stock_company_profile("SUNE", country='united states')

RuntimeError: ERR#0018: stock sune not found, check if it is correct.

In [106]:
data = investpy.stocks.get_stock_historical_data(stock='RDC', country='united states',
                                                 from_date='01/01/2014', to_date='01/01/2015').reset_index()

RuntimeError: ERR#0018: stock rdc not found, check if it is correct.

In [77]:
data

,Date,Open,High,Low,Close,Volume,Currency
0,2014-01-02,24.57,24.67,24.12,24.35,415743,USD
1,2014-01-03,24.37,24.96,24.12,24.20,699402,USD
2,2014-01-06,24.51,24.64,23.68,23.97,644066,USD
3,2014-01-07,24.05,24.37,23.55,23.76,942681,USD
4,2014-01-08,23.71,24.02,23.63,23.96,537299,USD
...,...,...,...,...,...,...,...
247,2014-12-24,24.47,24.56,24.02,24.46,95079,USD
248,2014-12-26,24.47,24.48,24.09,24.23,142224,USD
249,2014-12-29,24.25,24.65,24.11,24.38,177498,USD
250,2014-12-30,24.25,24.87,24.06,24.64,172024,USD


---

# TRY3 - combination of existing sp500 list and updates

In [607]:
with open("app/helper_files/sp500_constituents.json", 'r') as file:
    reference = json.load(file)

In [608]:
print("this data has {} changes.".format(len(reference.keys())))

this data has 145 changes.


In [612]:
URL_NEW = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
# the fixed version of the sp500 is
# https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=1095558369
# if the wiki page format changes, some of the codes below might not work.
# in that case, use the fixed version of the wiki page
page_NEW = requests.get(URL_NEW)

soup_NEW = BeautifulSoup(page_NEW.content, "html.parser")

changes = pd.read_html(str(soup_NEW.find(id='changes')))[0]
# concat the multi-headers into single header
changes.columns = ['_'.join(i) for i in list(changes.columns)]
changes.Date_Date = pd.to_datetime(changes.Date_Date, format="%B %d, %Y")

# this is the current list of sp500 companies
constituents = pd.read_html(str(soup_NEW.find(id='constituents')))[0]# change date format


In [613]:
construct = reference['2019/02/27']

changes_recent = changes.loc[changes.Date_Date > '2019-02-28']
changes_recent = changes_recent.sort_values(by='Date_Date')
current = constituents.Symbol.tolist()

for i, (d, new, _, old, _, _) in changes_recent.iterrows():
    
    print(d, new, old)

    try:
        construct = update_sp500_list(construct, new, old)
    except Exception as e:
        print("ERROR:{}. Trying to add {} and remove {}".format(d, new, old))
        print(e)
        
#         break
    print(len(construct))

2019-04-02 00:00:00 DOW BHF
509
2019-06-03 00:00:00 CTVA FLR
509
2019-06-03 00:00:00 DD DWDP
ERROR:2019-06-03 00:00:00. Trying to add DD and remove DWDP
list.remove(x): x not in list
509
2019-06-07 00:00:00 AMCR MAT
509
2019-07-01 00:00:00 MKTX LLL
509
2019-07-15 00:00:00 TMUS RHT
509
2019-08-09 00:00:00 IEX FL
509
2019-08-09 00:00:00 LDOS APC
509
2019-09-23 00:00:00 CDW TSS
509
2019-09-26 00:00:00 NVR JEF
ERROR:2019-09-26 00:00:00. Trying to add NVR and remove JEF
list.remove(x): x not in list
509
2019-10-03 00:00:00 LVS NKTR
509
2019-11-21 00:00:00 NOW CELG
509
2019-12-05 00:00:00 WRB VIAB
509
2019-12-09 00:00:00 ODFL STI
509
2019-12-23 00:00:00 LYV AMG
509
2019-12-23 00:00:00 STE MAC
509
2019-12-23 00:00:00 ZBRA TRIP
509
2020-01-28 00:00:00 PAYC WCG
509
2020-03-02 00:00:00 IR XEC
509
2020-04-01 00:00:00 HWM ARNC
509
2020-04-03 00:00:00 CARR nan
510
2020-04-03 00:00:00 OTIS nan
511
2020-04-06 00:00:00 nan RTN
510
2020-04-06 00:00:00 nan M
509
2020-05-12 00:00:00 DXCM AGN
509
2020-05-

In [580]:
reference.keys()

dict_keys(['2008/01/31', '2008/02/29', '2008/03/31', '2008/04/30', '2008/05/30', '2008/06/30', '2008/07/31', '2008/08/29', '2008/09/30', '2008/10/31', '2008/11/28', '2008/12/31', '2009/01/30', '2009/02/27', '2009/03/31', '2009/04/30', '2009/05/29', '2009/06/30', '2009/07/31', '2009/08/31', '2009/09/30', '2009/10/30', '2009/11/30', '2009/12/31', '2010/01/29', '2010/02/26', '2010/03/31', '2010/04/30', '2010/05/28', '2010/06/30', '2010/07/30', '2010/08/31', '2010/09/30', '2010/10/29', '2010/11/30', '2010/12/31', '2011/01/31', '2011/02/28', '2011/03/31', '2011/04/29', '2011/05/31', '2011/06/30', '2011/07/29', '2011/08/31', '2011/09/30', '2011/10/31', '2011/11/30', '2011/12/30', '2012/01/31', '2012/02/29', '2012/03/30', '2012/04/30', '2012/05/31', '2012/06/29', '2012/07/31', '2012/08/31', '2012/09/28', '2012/10/31', '2012/11/30', '2012/12/31', '2013/01/31', '2013/02/28', '2013/03/28', '2013/04/30', '2013/05/31', '2013/06/28', '2013/07/31', '2013/08/30', '2013/09/30', '2013/10/31', '2013/11/

In [291]:
update_dates = list(reference.keys())

for i, k in enumerate(update_dates):
    try:
        diff = set(reference[update_dates[i+1]]) - set(reference[update_dates[i]])
        print(diff)
    except Exception as e:
        print(k, e)

set()
{'HCP', 'PM'}
set()
set()
{'SWN', 'MEE', 'ISRG', 'LO', 'COG'}
{'MA', 'DVA', 'AKS', 'SNI'}
{'CF', 'IVZ'}
{'APH', 'PXD', 'CRM', 'HRS', 'BRIXX', 'FAST'}
{'FLS', 'NDAQ', 'DPS', 'WEC'}
{'XRAY', 'SRCL', 'SJM', 'PBCT', 'LIFE', 'WYNN', 'CEPH'}
{'EQT', 'MFE', 'DNB', 'RSG'}
{'OI', 'SCG', 'IRM', 'FLIR', 'HCN'}
{'BLKFDS', 'DO'}
{'HRL', 'ES', 'ORLY', 'VTR', 'TW'}
{'DNR'}
{'AVNW'}
{'DV', 'PCS', 'FTI'}
{'RHT', 'WDC', 'PWR'}
{'FMC'}
{'ARG', 'CFN'}
{'FSLR'}
{'PCLN'}
{'MJN', 'VISA', 'CLF', 'ROST', 'ROP', 'LDOS'}
{'NRG'}
{'BRKB', 'URBN'}
{'OKE', 'DISCA', 'HP'}
{'CERN'}
set()
{'KMX'}
{'QEP'}
{'TYC'}
set()
set()
{'IR'}
{'FFIV', 'CVC', 'NFX', 'NFLX'}
{'MMI', 'MSI', 'NE'}
{'JOY'}
{'COV', 'HII'}
{'EW', 'BLK', 'CMG'}
{'C'}
{'ANRZQ', 'PLD'}
{'ACN', 'MPN'}
set()
{'MOS'}
{'BEAM', 'TEL'}
{'XYL', 'CBE'}
{'OSHWQ', 'TRIP', 'BWA', 'OSHSQ', 'DLTR', 'PRGO'}
{'COV', 'WPX'}
set()
{'CCI'}
{'FOSL', 'XS'}
{'PSX', 'ALXN', 'KMI', 'KMIWS'}
{'LRCX', 'MNST'}
{'STX', 'ESV'}
set()
{'LYB'}
{'PETM', 'KRFT', 'ADT', 'KTF', 'PNR'}

## reference2

In [343]:
df = pd.read_csv('notebooks/sp500-master/S&P 500 Historical Components & Changes(07-05-2022).csv')

In [ ]:
df

In [352]:
df.loc[(df.date=='2007-07-02') | (df.date=='2007-06-26')]

,date,tickers
1289,2007-07-02,"A,AABA,AAPL,ABC,ABI,ABKFQ,ABT,ACS,ADBE,ADI,ADM..."


In [368]:
set(df.loc[df.date=='2007-07-02', 'tickers'].values[0].split(',')) - set(df.loc[df.date=='2007-06-26', 'tickers'].values[0].split(','))

{'COV', 'DFS', 'GGP', 'TEL'}

In [369]:
set(df.loc[df.date=='2007-06-26', 'tickers'].values[0].split(',')) - set(df.loc[df.date=='2007-07-02', 'tickers'].values[0].split(','))

{'ADCT', 'MEL', 'PMCS', 'SANM'}

In [367]:
'GGP' in df.loc[df.date=='2007-07-02', 'tickers'].values[0].split(',')

True

In [353]:
df.iloc[[1288, 1289]]

,date,tickers
1288,2007-06-26,"A,AABA,AAPL,ABC,ABI,ABKFQ,ABT,ACS,ADBE,ADCT,AD..."
1289,2007-07-02,"A,AABA,AAPL,ABC,ABI,ABKFQ,ABT,ACS,ADBE,ADI,ADM..."


In [345]:
df.loc[df.tickers.str.contains('GGP')]

,date,tickers
1289,2007-07-02,"A,AABA,AAPL,ABC,ABI,ABKFQ,ABT,ACS,ADBE,ADI,ADM..."
1290,2007-07-03,"A,AABA,AAPL,ABC,ABI,ABKFQ,ABT,ACS,ADBE,ADI,ADM..."
1291,2007-07-05,"A,AABA,AAPL,ABC,ABI,ABKFQ,ABT,ACS,ADBE,ADI,ADM..."
1292,2007-07-06,"A,AABA,AAPL,ABC,ABI,ABKFQ,ABT,ACS,ADBE,ADI,ADM..."
1293,2007-07-09,"A,AABA,AAPL,ABC,ABI,ABKFQ,ABT,ACAS,ACS,ADBE,AD..."
...,...,...
2543,2018-08-03,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,..."
2544,2018-08-06,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,..."
2545,2018-08-07,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,..."
2546,2018-08-17,"A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,..."


In [344]:
df.head()

,date,tickers
0,1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1,1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
2,1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
3,1996-01-10,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
4,1996-01-11,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."


### 2. Identify min and max date for each tickers

`ticker_list`: All the tickers that was/is in S&P500

In [4]:
ticker_list = set()

for s in sp500.tickers:
    s = set(s.split(","))
    ticker_list.update(s)
    
ticker_list = sorted(ticker_list)

Go through sp500 list and identify the min and max dates it was in S&P500

In [5]:
# convert into list format for easy comparison
sp500['tickers_list_format'] = sp500['tickers'].str.split(',')

In [6]:
# identify which date each ticker was included in sp500

ticker_dict = dict()
today = date.today().strftime("%Y-%m-%d")

# for each ticker, create a column where it indicates whether it is in the ticker list
# 1 if it is, 0 if not
# use this column to indicate the min and max date of the stock
for t in tqdm(ticker_list):
    found = list()
    for i in range(sp500.shape[0]):
        found.append(len(np.where(np.array(sp500['tickers_list_format'][i]) == t)[0]))
    sp500['found'] = found
    
    if sp500.loc[sp500['found']!=0, 'date'].max() == sp500['date'].max():
        max_date = today
    else:
        max_date = sp500.loc[sp500['found']!=0, 'date'].max()
    
    ticker_dict[t] = {'min_date' : sp500.loc[sp500['found']!=0, 'date'].min(),
                      'max_date' : max_date,
                      'dates' : sp500.loc[sp500['found']!=0, 'date'].values.tolist()}

  0%|          | 0/1128 [00:00<?, ?it/s]

In [50]:
# retrieve data for each ticker, but 1 year before the min_date
# this is to make sure we have all the data to calculate the moving averages

if 'stocks' in locals():
    if stocks.shape[0]>0:
        pass
    else:
        stocks = pd.DataFrame()
else:
    stocks = pd.DataFrame()

if 'NA_tickers' in locals():
    pass
else:
    NA_tickers = list()
    
    
for k in tqdm(ticker_dict.keys()):
    
    if k in stocks.ticker.unique():
        pass
    else:
        # set min date as 1 year before the actual min date
        s = ticker_dict[k]['min_date'].split('-')
        s[0] = str(int(s[0])-1)
        start_date = '-'.join(s)
        end_date = ticker_dict[k]['max_date']

        try:
            df = investing_api('stock', k, start_date, end_date)
            time.sleep(random.randrange(5, 10)) # set to pause for a bit when calling an api
        except Exception as e:
            if "ERR#0018" in str(e):
                try:
                    # retrieve data from alpha vintage
                    df = alpha_vintage_api('stock', k)
                    time.sleep(random.randrange(5, 10)) # set to pause for a bit when calling an api
                except:
                    NA_tickers.append(k)
                    print("Error when calling data for {}".format(k))

        stocks = pd.concat([stocks, df], axis=0)

  0%|          | 0/1128 [00:00<?, ?it/s]

Error when calling data for ABS
Error when calling data for ABX


2022-07-06 01:27:29,752 [app.utils.fetch_data] [INFO] Fetching stock from alpha vintage api: ACAS


Error when calling data for ACKH
Error when calling data for ACS
Error when calling data for ACV


2022-07-06 01:27:41,603 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: ADBE, from 06/05/1996 to 06/07/2022


...

Error when calling data for FBO


2022-07-06 02:31:47,140 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FCN, from 02/01/1995 to 01/10/1998
2022-07-06 02:31:57,392 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FCPT, from 10/11/2014 to 10/11/2015
2022-07-06 02:32:07,965 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FCX, from 02/01/1995 to 06/07/2022


Error when calling data for FDO


2022-07-06 23:06:02,111 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FDS, from 20/12/2020 to 06/07/2022
2022-07-06 23:06:10,548 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FDX, from 02/01/1995 to 06/07/2022
2022-07-06 23:06:17,906 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FE, from 02/01/1995 to 06/07/2022
2022-07-06 23:06:24,278 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FFIV, from 20/12/2009 to 06/07/2022
2022-07-06 23:06:37,383 [app.utils.fetch_data] [INFO] Fetching stock from alpha vintage api: FG
2022-07-06 23:06:45,641 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FHN, from 06/05/2001 to 19/06/2013
2022-07-06 23:06:52,678 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FII, from 16/04/2002 to 28/12/2012
2022-07-06 23:07:00,084 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: FIS, from 10/11/2005 to 06/07/2022
2022-07-06 23:07:10,889 [app.utils.f

KeyboardInterrupt: 

In [266]:
# set(NA_tickers)

In [17]:
import requests

In [12]:
with open('app/keys/keys.json', 'r') as key_file:
    keys = json.load(key_file)

In [14]:
key = keys['alpha_vintage']

In [475]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&apikey={}&outputsize=full'.format("Q", key)
r = requests.get(url)
data2 = r.json()

In [481]:
data2 = pd.DataFrame(data2['Time Series (Daily)'])
data2.T

,1. open,2. high,3. low,4. close,5. volume
2017-11-14,102.5800,103.6600,101.6200,103.4800,1299753
2017-11-13,101.9800,103.8700,101.1800,102.9100,1532499
2017-11-10,103.7000,104.6100,103.5700,104.0500,1138184
2017-11-09,103.5500,103.7700,101.5000,103.5100,2691961
2017-11-08,108.5200,108.8200,103.8200,104.7200,3089197
...,...,...,...,...,...
2013-05-15,45.2000,45.7400,44.8500,45.5000,2380500
2013-05-14,44.2400,44.7490,43.9000,44.4300,2049800
2013-05-13,42.3000,43.3700,42.3000,43.3200,1053100
2013-05-10,42.2600,42.9700,41.7800,42.3900,2969700


In [331]:
data.keys()

dict_keys(['Meta Data', 'Time Series (Daily)'])

In [56]:
investpy.stocks.get_stock_company_profile("ACI", country='united states')

RuntimeError: ERR#0018: stock aci not found, check if it is correct.

In [549]:
df = investing_api('stock', 'IQV', '2010-01-01', '2020-01-01')
df

2022-07-08 02:33:42,385 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: IQV, from 01/01/2010 to 01/01/2020


,Date,Open,High,Low,Close,Volume,Currency,ticker,type,p_key
0,2013-05-09,43.76,44.33,42.00,42.11,19154786,USD,IQV,stock,2013_05_09_IQV
1,2013-05-10,42.26,42.97,41.78,42.39,2969690,USD,IQV,stock,2013_05_10_IQV
2,2013-05-13,42.30,43.37,42.30,43.32,1053020,USD,IQV,stock,2013_05_13_IQV
3,2013-05-14,44.24,44.75,43.90,44.43,2051983,USD,IQV,stock,2013_05_14_IQV
4,2013-05-15,45.20,45.74,44.85,45.50,2381902,USD,IQV,stock,2013_05_15_IQV
...,...,...,...,...,...,...,...,...,...,...
1669,2019-12-24,155.03,155.12,154.04,154.70,483369,USD,IQV,stock,2019_12_24_IQV
1670,2019-12-26,155.00,155.00,153.45,154.14,1307260,USD,IQV,stock,2019_12_26_IQV
1671,2019-12-27,154.65,155.18,154.25,154.44,873921,USD,IQV,stock,2019_12_27_IQV
1672,2019-12-30,154.47,154.50,152.21,153.45,897936,USD,IQV,stock,2019_12_30_IQV
